# Export images and annotations from Datature Nexus using Datature Python SDK

In [ ]:
#!/usr/bin/env python
# -*-coding:utf-8 -*-
"""
  ████
██    ██   Datature
  ██  ██   Powering Breakthrough AI
    ██
 
@File    :   export_images_annotations.ipynb
@Author  :   Wei Loon Cheng
@Version :   1.0
@Contact :   hello@datature.io
@License :   Apache License 2.0
@Desc    :   Export images and annotations from Datature Nexus using Python SDK
"""

### Install prerequisites

In [1]:
%pip install datature
%pip install wget

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import time
import urllib
import wget
from zipfile import ZipFile

import datature

In [3]:
# change this to your project secret key
# it can be located on Nexus under the Integrations tab in your project page
datature.secret_key = "<YOUR_SECRET_KEY>"

### Download images

Images from your Nexus project will be downloaded and saved to the `images` folder. Datature Python SDK has an asset pagination limit of 100, so multiple calls will made to the API to download all images if your project has more than 100 images.

In [4]:
# change this to your desired image directory
image_dir = "images"
os.makedirs(image_dir, exist_ok=True)

# Download images from Nexus
print("Downloading images from Nexus...")
urls = []
assets = datature.Asset.list({"limit": 100})
urls.extend([(asset["filename"], asset["url"]) for asset in assets["data"]])
next_page = assets["next_page"] if "next_page" in assets else None

while next_page is not None:
    assets = datature.Asset.list({
        "limit": 100,
        "page": next_page,
    })
    urls.extend([(asset["filename"], asset["url"]) for asset in assets["data"]])
    next_page = assets["next_page"] if "next_page" in assets else None

for filename, url in urls:
    urllib.request.urlretrieve(url, os.path.join(image_dir, filename))
print(f"{len(urls)} images saved to: {image_dir}")

600 images saved to: images


### Export and download annotations

Datature supports the annotation formats listed [here](https://developers.datature.io/docs/exporting-annotations#annotation-formats). You can choose various formats depending on whether your annotations are bounding boxes or polygons.

In [5]:
# Download polygon annotation from Nexus
print("Downloading annotations from Nexus...")
operation = datature.Annotation.export(
    # change this to your desired annotation format
    # supported formats are listed here
    # https://developers.datature.io/docs/types-sdk-functions#annotationformat
    "polygon_coco",
    {
        "normalized": False,
        "shuffle": False,
        # 0.0 means all annotations are stored in the same file,
        # change this value if you want a train-test split
        "split_ratio": 0.0,
        "seed": 1,
    },
    background=True)

while datature.Operation.retrieve(
        operation["op_link"]
)["status"]["progress"]["with_status"]["finished"] != 1:
    time.sleep(1)

annotation_url = datature.Annotation.retrieve_exported_file(
    operation["id"])["download"]["url"]
annotation_zipfile = wget.download(annotation_url)

with ZipFile(annotation_zipfile, 'r') as zObject:
    annotation_filename = zObject.namelist()[-1]
    zObject.extractall()
print(f"Annotations saved to: {annotation_filename}")

Annotations saved to: 67f999395706c26d81b05524176f3258/dump.json
